PSD is computed using both non-overlapping segments and Welch method. The algorithm is repeated for both epileptic and non-epileptic group.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import welch
import pandas as pd

In [ ]:
def calculate_fft(data):
    fs=500
    # Calcola la trasformata di Fourier
    Y = np.fft.rfft(data)

    # Calcola le frequenze associate
    fft_freq = np.fft.rfftfreq(len(data), d=1/fs)

    # Calcola le ampiezze
    fft_amp = np.abs(Y) / len(data)

    # Calcola le fasi
    fft_phase = np.angle(Y)
    return fft_freq, fft_amp

def compute_power_spectrum(data):
    #2 sec
    Fs = 500
    N = 2/ (1/Fs)#al numeratore deve esserci la finestra temporale che si vuole considerare in secondi
    divided_arrays = np.array_split(data, len(data) // N)
    power_spectrum = np.zeros(int((len(divided_arrays[0]))/2)+1)
    for i in range(len(divided_arrays)):
        power_spectrum = power_spectrum + (np.abs(np.fft.rfft(divided_arrays[i])))**2
        # Get the list of frequencies for the components in fft_vals
    power_spectrum = power_spectrum/(len(divided_arrays))
    fft_freq = np.fft.rfftfreq(len(divided_arrays[0]), 1.0/Fs)
    return fft_freq,power_spectrum

def plot_psd(xout, fs):
    win = int(np.ceil(2 * fs))
    nfft = win
    noverlap_pc = [0.90]
    noverlap = np.ceil(win * np.array(noverlap_pc)).astype(int)
    col = ['r']

    plt.figure()

    
    f, pxx = welch(xout, fs, window='hann', nperseg=win, noverlap=noverlap, nfft=nfft)
    plt.semilogy(f, pxx, col)
    plt.grid(True)
    plt.legend([f'{pc * 100} %' for pc in noverlap_pc], fontsize=24)
    plt.xlabel('Frequency [Hz]', fontsize=12)
    plt.ylabel('Log-PSD [µV$^2$/Hz]', fontsize=12)
    plt.title('PSD using Welch method for varying overlap percentages', fontsize=24)
    plt.show()
    return (f,pxx)

In [ ]:
Fs = 500.0
Ts = 1.0/Fs
t = np.arange(len(colonna[0])) / Fs
#itero su tutti i canali
psd= []
psd_welch=[]
# Create a new folder for modified CSV files
new_folder = 'psd_non_epi'
new_folder_1='psd_welch_non_epi'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)
if not os.path.exists(new_folder_1):
    os.makedirs(new_folder_1)
# Itera su ogni array all'interno di "colonna"

for data,label_array in zip(colonna,labels_array):
    print('CHANNEL',label_array)

#grafici DATI FILTRATI
    fft_freqs,fft_vals=calculate_fft(data)
    plt.plot(fft_freqs, fft_vals)
    plt.semilogy()
    plt.show()

#POWER SPECTRL DENSITY
    fft_freq,power_spectrum=compute_power_spectrum(data)
    psd.append(power_spectrum)

# Plot the spectrum
    plt.plot(fft_freq,power_spectrum)
    plt.title('PSD filtered data')
    plt.xlabel('Frequency [Hz]', fontsize=12)
    plt.ylabel('Log-PSD [µV$^2$/Hz]', fontsize=12)
    plt.semilogy()
    plt.show()
    # Crea un nuovo DataFrame per la PSD
    psd_df = pd.DataFrame({
        'Frequenza': fft_freq,
        'PSD': power_spectrum
    })

    # Salva la PSD in un nuovo file CSV nella nuova cartella
    psd_df.to_csv(f'{new_folder}/{label_array}.csv', index=False)

    f,pxx=plot_psd(data,Fs)
    psd_welch.append(pxx)
    psd_welch_df = pd.DataFrame({
        'Frequenza': f,
        'PSD': pxx
    })

    # Salva la PSD in un nuovo file CSV nella nuova cartella
    psd_welch_df.to_csv(f'{new_folder_1}/{label_array}.csv', index=False)


